In [3]:
import requests 
import pandas as pd
import json
import matplotlib.pyplot as plt
import itertools
import collections 
import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import requests
from bs4 import BeautifulSoup

### API: https://developers.teleport.org/api/
### API ENDPOINT CREATOR: https://developers.teleport.org/api/reference/
### Embedded protocol: city:search-results/city:item/city:urban_area/ua:scores

### BRING IN THE DATA

In [4]:
city_stats_api='https://api.teleport.org/api/cities/?search=san%20diego%2C%20california&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores'
response=requests.get(city_stats_api)
R=response.json()
R

{'_embedded': {'city:search-results': [{'_embedded': {'city:item': {'_embedded': {'city:urban_area': {'_embedded': {'ua:scores': {'_links': {'self': {'href': 'https://api.teleport.org/api/urban_areas/slug:san-diego/scores/'}},
          'categories': [{'color': '#f3c32c',
            'name': 'Housing',
            'score_out_of_10': 2.9335},
           {'color': '#f3d630',
            'name': 'Cost of Living',
            'score_out_of_10': 5.156000000000001},
           {'color': '#f4eb33', 'name': 'Startups', 'score_out_of_10': 8.3505},
           {'color': '#d2ed31',
            'name': 'Venture Capital',
            'score_out_of_10': 8.595},
           {'color': '#7adc29',
            'name': 'Travel Connectivity',
            'score_out_of_10': 2.6220000000000003},
           {'color': '#36cc24', 'name': 'Commute', 'score_out_of_10': 4.40975},
           {'color': '#19ad51',
            'name': 'Business Freedom',
            'score_out_of_10': 8.671},
           {'color': '#0d69

### CREATE A DATA FRAME

In [5]:
pd.DataFrame(R) 

,_embedded,_links,count
city:search-results,[{'_embedded': {'city:item': {'_embedded': {'c...,NaN,2
curies,NaN,[{'href': 'https://developers.teleport.org/api...,2
self,NaN,{'href': 'https://api.teleport.org/api/cities/...,2


### TOO NESTED. PULL IT OUT!

In [6]:
Re1=R['_embedded']
RE1=pd.json_normalize(Re1)
RE1
Rcs=RE1['city:search-results']
RCS=pd.json_normalize(Rcs)
RCS
Rman=RCS[1]
RMAN=pd.json_normalize(Rman)
RMAN

,matching_alternate_names,matching_full_name,_embedded.city:item._embedded.city:urban_area._embedded.ua:scores._links.self.href,_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.categories,_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.summary,_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.teleport_city_score,_embedded.city:item._embedded.city:urban_area._links.self.href,_embedded.city:item._embedded.city:urban_area._links.ua:admin1-divisions,_embedded.city:item._embedded.city:urban_area._links.ua:cities.href,_embedded.city:item._embedded.city:urban_area._links.ua:continent.href,...,_embedded.city:item._links.city:urban_area.name,_embedded.city:item._links.self.href,_embedded.city:item.full_name,_embedded.city:item.geoname_id,_embedded.city:item.location.geohash,_embedded.city:item.location.latlon.latitude,_embedded.city:item.location.latlon.longitude,_embedded.city:item.name,_embedded.city:item.population,_links.city:item.href
0,[{'name': 'San Diego Country Estates'}],"San Diego Country Estates, California, United ...",https://api.teleport.org/api/urban_areas/slug:...,"[{'color': '#f3c32c', 'name': 'Housing', 'scor...","<p>San Diego, California, is among the top cit...",64.593108,https://api.teleport.org/api/urban_areas/slug:...,[{'href': 'https://api.teleport.org/api/countr...,https://api.teleport.org/api/urban_areas/slug:...,https://api.teleport.org/api/continents/geonam...,...,San Diego,https://api.teleport.org/api/cities/geonameid:...,"San Diego Country Estates, California, United ...",5391831,9mugy88wpbww2q1m09zg,33.00671,-116.78364,San Diego Country Estates,10109,https://api.teleport.org/api/cities/geonameid:...


In [7]:
RMAN.keys()

Index(['matching_alternate_names', 'matching_full_name',
       '_embedded.city:item._embedded.city:urban_area._embedded.ua:scores._links.self.href',
       '_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.categories',
       '_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.summary',
       '_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.teleport_city_score',
       '_embedded.city:item._embedded.city:urban_area._links.self.href',
       '_embedded.city:item._embedded.city:urban_area._links.ua:admin1-divisions',
       '_embedded.city:item._embedded.city:urban_area._links.ua:cities.href',
       '_embedded.city:item._embedded.city:urban_area._links.ua:continent.href',
       '_embedded.city:item._embedded.city:urban_area._links.ua:continent.name',
       '_embedded.city:item._embedded.city:urban_area._links.ua:countries',
       '_embedded.city:item._embedded.city:urban_area._links.ua:details.href',
       '_embedded.city:item._emb

In [8]:
Rscores=RMAN['_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.categories']
RSCORES=pd.json_normalize(Rscores)
RSCORES

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,"{'color': '#f3c32c', 'name': 'Housing', 'score...","{'color': '#f3d630', 'name': 'Cost of Living',...","{'color': '#f4eb33', 'name': 'Startups', 'scor...","{'color': '#d2ed31', 'name': 'Venture Capital'...","{'color': '#7adc29', 'name': 'Travel Connectiv...","{'color': '#36cc24', 'name': 'Commute', 'score...","{'color': '#19ad51', 'name': 'Business Freedom...","{'color': '#0d6999', 'name': 'Safety', 'score_...","{'color': '#051fa5', 'name': 'Healthcare', 'sc...","{'color': '#150e78', 'name': 'Education', 'sco...","{'color': '#3d14a4', 'name': 'Environmental Qu...","{'color': '#5c14a1', 'name': 'Economy', 'score...","{'color': '#88149f', 'name': 'Taxation', 'scor...","{'color': '#b9117d', 'name': 'Internet Access'...","{'color': '#d10d54', 'name': 'Leisure & Cultur...","{'color': '#e70c26', 'name': 'Tolerance', 'sco...","{'color': '#f1351b', 'name': 'Outdoors', 'scor..."


In [9]:
super_json=[]
for i in range(17):
    score=RSCORES[i]
    super_json.append(score)

In [10]:
super_json
df=pd.DataFrame(super_json)
DATA=df[0]
data=pd.json_normalize(DATA)
SD_=data[['name','score_out_of_10']]
SD_

,name,score_out_of_10
0,Housing,2.933500
1,Cost of Living,5.156000
2,Startups,8.350500
3,Venture Capital,8.595000
4,Travel Connectivity,2.622000
5,Commute,4.409750
6,Business Freedom,8.671000
7,Safety,6.933000
8,Healthcare,8.767667
9,Education,5.457500


### GREAT NOW LET'S LOOK AT LOW SCORES

In [37]:
SD=SD_.sort_values(by="score_out_of_10")
SD

,name,score_out_of_10
4,Travel Connectivity,2.622000
0,Housing,2.933500
5,Commute,4.409750
12,Taxation,4.488000
1,Cost of Living,5.156000
9,Education,5.457500
13,Internet Access,5.999000
16,Outdoors,6.396000
11,Economy,6.514500
7,Safety,6.933000


### CREATE A FUNCTION THAT WILL PRODUCE THE INDEX PER CITY. 

In [12]:
def city_scores(ulr): 
    city_stats_api=ulr
    response=requests.get(city_stats_api)
    R=response.json()
    pd.DataFrame(R)
    Re1=R['_embedded']
    RE1=pd.json_normalize(Re1)
    Rcs=RE1['city:search-results']
    RCS=pd.json_normalize(Rcs)
    Rman=RCS[1]
    RMAN=pd.json_normalize(Rman)
    Rscores=RMAN['_embedded.city:item._embedded.city:urban_area._embedded.ua:scores.categories']
    RSCORES=pd.json_normalize(Rscores)
    super_json=[]
    for i in range(17):
        score=RSCORES[i]
        super_json.append(score)
    df=pd.DataFrame(super_json)
    DATA=df[0]
    data=pd.json_normalize(DATA)
    indices=data[['name','score_out_of_10']]
    return indices

### OTHER CITIES URLs
CDMX: https://api.teleport.org/api/cities/?search=Mexico%20City%20&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores


In [13]:
CDMX_=city_scores('https://api.teleport.org/api/cities/?search=Mexico%20City%20&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
CDMX=CDMX_.sort_values(by="score_out_of_10")
CDMX

,name,score_out_of_10
9,Education,1.713000
10,Environmental Quality,2.506250
3,Venture Capital,2.634000
4,Travel Connectivity,2.870000
11,Economy,3.500500
5,Commute,3.806000
16,Outdoors,3.914000
12,Taxation,4.333500
7,Safety,5.464000
6,Business Freedom,5.761000


### INTERESTING WAYS TO LOOK AT THIS INFO:
In this example we will look at two cities separately. But there are many things we could do now we've created a "generator" function.

-We could create an aggregate database with the information of all major cities in North America (or any group of cities really. Whatever tickles your fancy). 

-We could compare every category and see if it fits a standard distribution. We could calculate standard deviations for every city in every category.

But for now we'll create that function first...

### EXAMPLE OF AGGREGATE DATA FRAME


In [14]:
CHICAGO=city_scores('https://api.teleport.org/api/cities/?search=chicago&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
SF=city_scores('https://api.teleport.org/api/cities/?search=san%20francisco&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
NY= city_scores('https://api.teleport.org/api/cities/?search=new%20york&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
MIAMI=city_scores('https://api.teleport.org/api/cities/?search=miami&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
GDL=city_scores('https://api.teleport.org/api/cities/?search=guadalajara&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
VAN=city_scores('https://api.teleport.org/api/cities/?search=vancouver&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
MONTR=city_scores('https://api.teleport.org/api/cities/?search=montreal&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')
QUEBEC=city_scores('https://api.teleport.org/api/cities/?search=quebec&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores')


In [15]:
from functools import reduce
data_frames=[CHICAGO,SF,NY,MIAMI,GDL,VAN,MONTR,QUEBEC,CDMX_,SD_]
cities_merged = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
                                            how='outer'), data_frames)
cities_merged.columns=['CATEGORY','CHICAGO','SF','NY','MIAMI','GDL','VAN','MONTR','QUEBEC','CDMX','SD']
cities_merged


/var/folders/51/zl81hkbj365fyht29sw2pjsw0000gn/T/ipykernel_38245/1988991041.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'score_out_of_10_x'} in the result is deprecated and will raise a MergeError in a future version.
  cities_merged = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
/var/folders/51/zl81hkbj365fyht29sw2pjsw0000gn/T/ipykernel_38245/1988991041.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'score_out_of_10_x'} in the result is deprecated and will raise a MergeError in a future version.
  cities_merged = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
/var/folders/51/zl81hkbj365fyht29sw2pjsw0000gn/T/ipykernel_38245/1988991041.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'score_out_of_10_x'} in the result is deprecated and will raise a MergeError in a future version.
  cities_merged = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
/var/folders/51/zl81hkbj365fyht

,CATEGORY,CHICAGO,SF,NY,MIAMI,GDL,VAN,MONTR,QUEBEC,CDMX,SD
0,Housing,3.70200,9.016500,1.000000,3.03100,6.414000,4.138000,7.392000,7.392000,8.360000,2.933500
1,Cost of Living,4.76600,7.822000,2.342000,4.49700,6.249000,4.924000,5.948000,5.948000,9.244000,5.156000
2,Startups,9.51300,2.857500,10.000000,8.02250,0.000000,6.957000,8.102500,8.102500,7.480000,8.350500
3,Venture Capital,8.36800,0.000000,10.000000,5.05500,4.790000,5.899000,5.773000,5.773000,2.634000,8.595000
4,Travel Connectivity,5.50750,0.890000,6.675000,4.16900,5.855500,2.240500,3.443000,3.443000,2.870000,2.622000
5,Commute,4.57975,4.789250,5.519250,4.05275,5.285750,4.908000,5.107000,5.107000,3.806000,4.409750
6,Business Freedom,8.67100,3.021333,8.671000,8.67100,7.119667,8.671000,8.966000,8.966000,5.761000,8.671000
7,Safety,3.83200,6.969000,7.022000,4.32100,7.792500,5.180500,7.822000,7.822000,5.464000,6.933000
8,Healthcare,8.55500,5.324000,8.501667,8.52900,8.628333,8.596667,8.325667,8.325667,5.789667,8.767667
9,Education,7.99900,0.000000,8.093500,4.39450,4.291500,4.215500,7.299000,7.299000,1.713000,5.457500


In [16]:
CITIES=cities_merged.transpose().iloc[1: , :]
CITIES.columns=['Housing','Cost of Living','Startups','Venture Capital','Travel Connectivity','Commute','Business Freedom','Safety','Healthcare','Education','Environmental Quality','Economy','Taxation','Internet Access', 'Leisure & Culture','Tolerance','Outdoors']
CITIES['Name'] = CITIES.index
column_order=['Name','Housing','Cost of Living','Startups','Venture Capital','Travel Connectivity','Commute','Business Freedom','Safety','Healthcare','Education','Environmental Quality','Economy','Taxation','Internet Access', 'Leisure & Culture','Tolerance','Outdoors']
CITIES=CITIES[column_order]
CITIES


,Name,Housing,Cost of Living,Startups,Venture Capital,Travel Connectivity,Commute,Business Freedom,Safety,Healthcare,Education,Environmental Quality,Economy,Taxation,Internet Access,Leisure & Culture,Tolerance,Outdoors
CHICAGO,CHICAGO,3.702,4.766,9.513,8.368,5.5075,4.57975,8.671,3.832,8.555,7.999,6.8045,6.5145,4.062,6.5995,8.193,6.134,5.4155
SF,SF,9.0165,7.822,2.8575,0.0,0.89,4.78925,3.021333,6.969,5.324,0.0,4.17625,3.8855,7.514,3.4815,4.8015,8.8265,5.5
NY,NY,1.0,2.342,10.0,10.0,6.675,5.51925,8.671,7.022,8.501667,8.0935,5.23375,6.5145,3.9205,7.0985,10.0,6.7125,5.7475
MIAMI,MIAMI,3.031,4.497,8.0225,5.055,4.169,4.05275,8.671,4.321,8.529,4.3945,6.292,6.5145,4.772,5.824,6.7675,6.0435,5.3845
GDL,GDL,6.414,6.249,0.0,4.79,5.8555,5.28575,7.119667,7.7925,8.628333,4.2915,5.5655,4.296,3.894,6.241,9.338,7.665,6.22
VAN,VAN,4.138,4.924,6.957,5.899,2.2405,4.908,8.671,5.1805,8.596667,4.2155,8.4595,6.5145,3.9205,5.9685,6.2195,8.1595,6.304
MONTR,MONTR,7.392,5.948,8.1025,5.773,3.443,5.107,8.966,7.822,8.325667,7.299,7.7215,5.8405,7.2745,4.478,6.9175,8.193,5.303
QUEBEC,QUEBEC,7.392,5.948,8.1025,5.773,3.443,5.107,8.966,7.822,8.325667,7.299,7.7215,5.8405,7.2745,4.478,6.9175,8.193,5.303
CDMX,CDMX,8.36,9.244,7.48,2.634,2.87,3.806,5.761,5.464,5.789667,1.713,2.50625,3.5005,4.3335,5.8405,9.962,7.089,3.914
SD,SD,2.9335,5.156,8.3505,8.595,2.622,4.40975,8.671,6.933,8.767667,5.4575,7.26725,6.5145,4.488,5.999,8.21,8.5395,6.396


In [48]:
hs_labels=['It sucks balls',"I've seen worse",'Ok',"I'll tap that",'Awesome']
bins=pd.cut(CITIES['Housing'],bins=5,labels=hs_labels)
bins

CHICAGO    I've seen worse
SF                 Awesome
NY          It sucks balls
MIAMI      I've seen worse
GDL          I'll tap that
VAN        I've seen worse
MONTR        I'll tap that
QUEBEC       I'll tap that
CDMX               Awesome
SD         I've seen worse
Name: Housing, dtype: category
Categories (5, object): ['It sucks balls' < 'I've seen worse' < 'Ok' < 'I'll tap that' < 'Awesome']

In [18]:
healthcare=CITIES['Healthcare'].mean()
healthcare

7.934333333333332

In [19]:
cities=CITIES.to_csv('/Users/derinkivaner/Desktop/IH/PYTHON/WEB_PROJECT/cities.csv',index=False)

# LETS LOOK AT SD AND CDMX

In [45]:
CDMX

,name,score_out_of_10
9,Education,1.713000
10,Environmental Quality,2.506250
3,Venture Capital,2.634000
4,Travel Connectivity,2.870000
11,Economy,3.500500
5,Commute,3.806000
16,Outdoors,3.914000
12,Taxation,4.333500
7,Safety,5.464000
6,Business Freedom,5.761000


In [46]:
SD

,name,score_out_of_10
4,Travel Connectivity,2.622000
0,Housing,2.933500
5,Commute,4.409750
12,Taxation,4.488000
1,Cost of Living,5.156000
9,Education,5.457500
13,Internet Access,5.999000
16,Outdoors,6.396000
11,Economy,6.514500
7,Safety,6.933000


# WEB SCRAPING LET'S LOOK AT THESE CITIES ON WIKIPEDIA

How does wikipedia talk about the challenges of these two city?
take the three lowest scores and see if they are addressed at all in the wiki page. 

In [21]:
SD_low_scores=['travel','housing','commute']

In [22]:
ws_url='https://en.wikipedia.org/wiki/San_Diego'
page=requests.get(ws_url).content
page

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>San Diego - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"42487749-79a0-4ad0-ab71-ea590e31d6e4","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"San_Diego","wgTitle":"San Diego","wgCurRevisionId":1105295920,"wgRevisionId":1105295920,"wgArticleId":28504,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with non-numeric formatnum arguments","Webarchive template wayback links","All articles with dead external links","Articles with dead external links from 

In [23]:
soupSD= BeautifulSoup(page,'html')
sections= [i.text.replace('[edit]','').lower() for i in soupSD.find_all('h2')]
sections

['contents',
 'history',
 'geography',
 'demographics',
 'economy',
 'government',
 'education',
 'culture',
 'sports',
 'media',
 'infrastructure',
 'notable people',
 'sister cities',
 'notes',
 'references',
 'external links',
 'navigation menu']

In [24]:
for i in SD_low_scores:
    if i in sections:
        print(f'\u2705 {i} is addressed as a main section in wikipedia')
    else:
        print(f'\u274c {i} is NOT a main section in wikipedia')

❌ travel is NOT a main section in wikipedia
❌ housing is NOT a main section in wikipedia
❌ commute is NOT a main section in wikipedia


In [25]:
sub_sections= [i.text.replace('[edit]','').strip().lower() for i in soupSD.find_all('h3')]
sub_sections

['pre-colonial period',
 'spanish period',
 'mexican period',
 'american period',
 'communities and neighborhoods',
 'cityscape',
 'climate',
 'ecology',
 'defense and military',
 'tourism',
 'international trade',
 'companies',
 'top employers',
 'real estate',
 'local government',
 'state and federal representation',
 'election history',
 'major scandals',
 'crime',
 'primary and secondary schools',
 'colleges and universities',
 'libraries',
 'teams',
 'events',
 'utilities',
 'street lights',
 'transportation',
 'general sources',
 'personal tools',
 'namespaces',
 'views',
 'search',
 'navigation',
 'contribute',
 'tools',
 'print/export',
 'in other projects',
 'languages']

In [26]:
for i in SD_low_scores:
    if i in sub_sections:
        print(f'\u2705 {i} is addressed as a sub-section')
    else:
        print(f'\u274c {i} is NOT a sub-section')

❌ travel is NOT a sub-section
❌ housing is NOT a sub-section
❌ commute is NOT a sub-section


In [27]:
parragraphs= [i.text.replace('[edit]','').strip().lower() for i in soupSD.find_all('p')][1:]
parragraphs[0]


"san diego (/ˌsæn diˈeɪɡoʊ/ san dee-ay-goh, spanish:\xa0[san ˈdjeɣo]; spanish for 'saint didacus') is a major city in the u.s. state of california on the coast of the pacific ocean and immediately adjacent to the mexican border. with a 2020 population of 1,386,932,[11] san diego is the eighth most populous city in the united states and second most populous in california (after los angeles). the city is the county seat of san diego county, the fifth most populous county in the united states, with 3,338,330 estimated residents as of 2019. the city is known for its mild year-round climate, natural deep-water harbor, extensive beaches and parks, long association with the united states navy and marine corps, and recent emergence as a healthcare and biotechnology development center."

In [28]:
cleanP=[]
for i in parragraphs:
    j = re.sub('[^A-Za-z0-9]+', ' ', i)
    cleanP.append(j)
cleanP

['san diego s n di e o san dee ay goh spanish san dje o spanish for saint didacus is a major city in the u s state of california on the coast of the pacific ocean and immediately adjacent to the mexican border with a 2020 population of 1 386 932 11 san diego is the eighth most populous city in the united states and second most populous in california after los angeles the city is the county seat of san diego county the fifth most populous county in the united states with 3 338 330 estimated residents as of 2019 the city is known for its mild year round climate natural deep water harbor extensive beaches and parks long association with the united states navy and marine corps and recent emergence as a healthcare and biotechnology development center ',
 'historically home to the kumeyaay people san diego is frequently referred to as the birthplace of california as it was the first site visited and settled by europeans on what is now the west coast of the united states 12 upon landing in sa

In [29]:
split_parragraphs=[]
for i in cleanP:
    y=i.split(' ')
    split_parragraphs.append(y)
split_parragraphs

[['san',
  'diego',
  's',
  'n',
  'di',
  'e',
  'o',
  'san',
  'dee',
  'ay',
  'goh',
  'spanish',
  'san',
  'dje',
  'o',
  'spanish',
  'for',
  'saint',
  'didacus',
  'is',
  'a',
  'major',
  'city',
  'in',
  'the',
  'u',
  's',
  'state',
  'of',
  'california',
  'on',
  'the',
  'coast',
  'of',
  'the',
  'pacific',
  'ocean',
  'and',
  'immediately',
  'adjacent',
  'to',
  'the',
  'mexican',
  'border',
  'with',
  'a',
  '2020',
  'population',
  'of',
  '1',
  '386',
  '932',
  '11',
  'san',
  'diego',
  'is',
  'the',
  'eighth',
  'most',
  'populous',
  'city',
  'in',
  'the',
  'united',
  'states',
  'and',
  'second',
  'most',
  'populous',
  'in',
  'california',
  'after',
  'los',
  'angeles',
  'the',
  'city',
  'is',
  'the',
  'county',
  'seat',
  'of',
  'san',
  'diego',
  'county',
  'the',
  'fifth',
  'most',
  'populous',
  'county',
  'in',
  'the',
  'united',
  'states',
  'with',
  '3',
  '338',
  '330',
  'estimated',
  'residents',
  

In [30]:
travel=0
housing=0
commute=0
for i in split_parragraphs:
    for j in i:
            if j == SD_low_scores[0]:
                travel=travel+1
            elif j == SD_low_scores[1]:
                housing=housing+1
            elif j == SD_low_scores[2]:
                commute=commute+1
print(f'Travel was mentioned {travel} times in parragraphs.')
print(f'Housing was mentioned {housing} times in parragraphs.')
print(f'Commute was mentioned {commute} times in parragraphs.')

Travel was mentioned 0 times in parragraphs.
Housing was mentioned 2 times in parragraphs.
Commute was mentioned 0 times in parragraphs.


In [31]:
CDMX_low_scores=['education','environment','venture']
wsmx_url='https://en.wikipedia.org/wiki/Mexico_City'
pagemx=requests.get(wsmx_url).content
soupmx= BeautifulSoup(pagemx,'html')
sectionsmx= [i.text.replace('[edit]','').lower() for i in soupmx.find_all('h2')]
sub_sectionsmx= [i.text.replace('[edit]','').strip().lower() for i in soupmx.find_all('h3')]
parragraphsmx= [i.text.replace('[edit]','').strip().lower() for i in soupmx.find_all('p')][1:]
cleanPMX=[]
for i in parragraphsmx:
    jmx = re.sub('[^A-Za-z0-9]+', ' ', i)
    cleanPMX.append(jmx)
split_parragraphsMX=[]
for i in cleanPMX:
    ymx=i.split(' ')
    split_parragraphsMX.append(ymx)
split_parragraphsMX[0]

['mexico',
 'city',
 'spanish',
 'ciudad',
 'de',
 'm',
 'xico',
 'a',
 '12',
 'locally',
 'sju',
 'a',
 'e',
 'mexiko',
 'listen',
 '13',
 'abbr',
 'cdmx',
 'nahuatl',
 'altepetl',
 'mexico',
 'is',
 'the',
 'capital',
 'and',
 'largest',
 'city',
 'of',
 'mexico',
 'and',
 'the',
 'most',
 'populous',
 'city',
 'in',
 'north',
 'america',
 '14',
 '15',
 'it',
 'is',
 'one',
 'of',
 'the',
 'country',
 's',
 '32',
 'federal',
 'entities',
 'mexico',
 'city',
 'is',
 'one',
 'of',
 'the',
 'most',
 'important',
 'cultural',
 'and',
 'financial',
 'centers',
 'in',
 'the',
 'world',
 '16',
 'it',
 'is',
 'located',
 'in',
 'the',
 'valley',
 'of',
 'mexico',
 'in',
 'the',
 'high',
 'central',
 'plateau',
 'at',
 'an',
 'altitude',
 'of',
 '2',
 '240',
 'meters',
 '7',
 '350',
 'ft',
 'the',
 'city',
 'has',
 '16',
 'boroughs',
 'or',
 'demarcaciones',
 'territoriales',
 'which',
 'are',
 'in',
 'turn',
 'divided',
 'into',
 'neighborhoods',
 'or',
 'colonias',
 '']

In [32]:
for i in CDMX_low_scores:
    if i in sectionsmx:
        print(f'\u2705 {i} is addressed as a main section in wikipedia')
    else:
        print(f'\u274c {i} is NOT a main section in wikipedia')

❌ education is NOT a main section in wikipedia
❌ environment is NOT a main section in wikipedia
❌ venture is NOT a main section in wikipedia


In [33]:
for i in CDMX_low_scores:
    if i in sub_sectionsmx:
        print(f'\u2705 {i} is addressed as a sub-section')
    else:
        print(f'\u274c {i} is NOT a sub-section')  

✅ education is addressed as a sub-section
✅ environment is addressed as a sub-section
❌ venture is NOT a sub-section


In [34]:
education=0
environment=0
venture=0
for i in split_parragraphsMX:
    for j in i:
            if j == CDMX_low_scores[0]:
                education=education+1
            elif j == CDMX_low_scores[1]:
                environment=environment+1
            elif j == CDMX_low_scores[2]:
                venture=venture+1
print(f'Education was mentioned {education} times in parragraphs.')
print(f'Environment was mentioned {environment} times in parragraphs.')
print(f'Venture was mentioned {venture} times in parragraphs.')

Education was mentioned 6 times in parragraphs.
Environment was mentioned 0 times in parragraphs.
Venture was mentioned 0 times in parragraphs.


In [35]:
# education=0
# environment=0
# venture=0
# capital=0
# for i in CDMX_low_scores:
#     for j in split_parragraphsMX:
#         for k in j:
#             if k.startswith(i[0]):
#                 education=education+1
#             elif k.startswith(i[1]):
#                 environment=environment+1
#             elif k.startswith(i[2]):
#                 venture=venture+1
#             elif k.startswith(i[3]):
#                 capital=capital+1
# print(f'Education was mentioned {education} times in parragraphs.')
# print(f'Environment was mentioned {environment} times in parragraphs.')
# print(f'Venture was mentioned {venture} times in parragraphs.')
# print(f'Capital was mentioned {capital} times in parragraphs.')

### IDEAS TO IMPROVE 
-The scraping of the public image could be done directly on each governments/administration's page or twitter and on news papers headlines. 
-The word search should also look for other words with different endings. 
-The words could include related words i.e environment {pollution, draught, etc}